# Inhaled Corticosteroid codelists

## All Inhaled Corticosteroids

This list of AMPs is derived from the denominator of [OpenPrescribing high dose inhaled corticosteroid measure](https://github.com/ebmdatalab/openprescribing/blob/master/openprescribing/measure_definitions/icsdose.json) definition.

[All inhaled steroids](#all)

In [11]:
from ebmdatalab import bq
import os
import pandas as pd

In [12]:
#define high and med/low dose ICS inhaler variable for SQL search

high_dose_sql = """
CASE WHEN bnf_code LIKE '0302000C0%AC' THEN 'high' # Beclomet Diprop_Inha 250mcg (200D) (brands and generic)
WHEN bnf_code LIKE '0302000C0%AU' THEN 'high' # Beclomet Diprop_Inha B/A 250mcg (200 D) (brands and generic)
WHEN bnf_code LIKE '0302000C0%BK' THEN 'high' # Beclomet Diprop_Pdr For Inh 250mcg(100 D (brands and generic)
WHEN bnf_code LIKE '0302000C0%BW' THEN 'high' # Beclomet Diprop_Inha 250mcg (200 D) CFF (brands and generic)
WHEN bnf_code LIKE '0302000C0%BZ' THEN 'high' # Beclomet/Formoterol_Inh 200/6 (120D) CFF (brands and generic)
WHEN bnf_code LIKE '0302000C0%CA' THEN 'high' # Beclomet/Formoterol_Inh 200/6 (120D) Dry (brands and generic)
WHEN bnf_code LIKE '0302000K0%AH' THEN 'high' # Budesonide_Pdr For Inh 400mcg (50 D) (brands and generic)
WHEN bnf_code LIKE '0302000K0%AY' THEN 'high' # Budesonide_Pdr For Inh 400mcg (100 D) (brands and generic)
WHEN bnf_code LIKE '0302000K0%AU' THEN 'high' # Budesonide/Formoterol_InhaB/A 400/12(60D (brands and generic)
WHEN bnf_code LIKE '0302000N0%AF' THEN 'high' # Fluticasone Prop_Pdr Inh 250mcg Disk Ref (brands and generic)
WHEN bnf_code LIKE '0302000N0%AP' THEN 'high' # Fluticasone Prop_Pdr Inh 500mcg Disk+Dev (brands and generic)
WHEN bnf_code LIKE '0302000N0%AU' THEN 'high' # Fluticasone Prop_Pdr For Inh 500mcg(60D) (brands and generic)
WHEN bnf_code LIKE '0302000N0%AZ' THEN 'high' # Fluticasone/Salmeterol_Inh 500/50mcg 60D (brands and generic)
WHEN bnf_code LIKE '0302000N0%BC' THEN 'high' # ...
WHEN bnf_code LIKE '0302000N0%BD' THEN 'high' # ...
WHEN bnf_code LIKE '0302000N0%BG' THEN 'high' # Fluticasone/Salmeterol_Inh 250/25mcg120D (brands and generic)
WHEN bnf_code LIKE '0302000N0%BK' THEN 'high' # Fluticasone/Formoterol_Inh 250/10mcg120D (brands and generic)
WHEN bnf_code LIKE '0302000U0%AB' THEN 'high' # Ciclesonide_Inh 160mcg (120 D) CFF (brands and generic)
WHEN bnf_code LIKE '0302000V0%AA' THEN 'high' # Fluticasone/Vilanterol_Inha 184/22mcg30D (brands and generic)"
ELSE 'med_low'
END AS strength
"""

In [13]:
#Single SQL for all different types of inhalers

sql = '''
WITH bnf_codes AS (
  SELECT 
    DISTINCT bnf_code 
  FROM 
    measures.dmd_objs_with_form_route 
  WHERE 
    (
      bnf_code LIKE '0302000C0%' OR #BNF Beclometasone dipropionate
      bnf_code LIKE '0301011AB%' OR #BNF BeclometDiprop/Formoterol/Glycopyrronium
      bnf_code LIKE '0302000K0%' OR #BNF budesonide
      bnf_code LIKE '0302000U0%' OR #BNF Ciclesonide
      bnf_code LIKE '0302000V0%' OR #BNF Fluticasone furoate
      bnf_code LIKE '0302000N0%' OR #BNF Fluticasone propionate
      bnf_code LIKE '0302000R0%'#BNF Mometasone Furoate
      ) 
    AND (
      form_route LIKE '%pressurizedinhalation.inhalation' 
      OR form_route LIKE 'powderinhalation.inhalation%'
    ) # this provides BNF codes for all inhalers (MDI and DPI) which contain corticosteroid
) 
SELECT 
  'vmp' AS type, 
  vmp.id AS id, #VMP from DM+D
  vmp.bnf_code AS bnf_code, 
  vmp.nm AS nm, #VMP name
  COUNT(ing) AS chem_count, #how many chemical substances in device
  {} #variable high_dose_sql, from code in above cell: describes inhalers as either "high" dose or "med/low"
FROM 
  dmd.vmp AS vmp 
  INNER JOIN dmd.vpi AS vpi ON vmp.id = vpi.vmp 
WHERE 
  bnf_code IN (
    SELECT 
      * 
    FROM 
      bnf_codes
  ) 
GROUP BY 
  vmp.id, 
  vmp.bnf_code, 
  vmp.nm 
UNION ALL 
SELECT 
  "amp" AS type, 
  amp.id AS id, 
  amp.bnf_code AS bnf_code, 
  amp.descr AS nm, 
  COUNT(ing) AS chem_count, 
  {} 
FROM 
  dmd.amp as amp 
  INNER JOIN dmd.vpi as vpi ON amp.vmp = vpi.vmp 
WHERE 
  bnf_code IN (
    SELECT 
      * 
    FROM 
      bnf_codes
  ) 
GROUP BY 
  amp.id, 
  amp.bnf_code, 
  amp.descr
'''

sql = sql.format(high_dose_sql,high_dose_sql) #adds in high_dose_ics variable
all_ics = bq.cached_read(sql, csv_path=os.path.join('..','data','new_all_ics_codelist.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
all_ics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   type        216 non-null    object
 1   id          216 non-null    int64 
 2   bnf_code    216 non-null    object
 3   nm          216 non-null    object
 4   chem_count  216 non-null    int64 
 5   strength    216 non-null    object
dtypes: int64(2), object(4)
memory usage: 10.2+ KB


## Single Agent Inhaled Corticosteroids

In [14]:
#to create a list of single agent ICS, we filter dataframe to where the number of chemical substances in the inhaler = 1
single_ics = all_ics.loc[all_ics['chem_count'] == 1]
single_ics = single_ics.drop(columns=["chem_count","strength"]) #drop the strength and chemical substance count from the table

single_ics

,type,id,bnf_code,nm
1,amp,3176811000001102,0302000C0BJAABE,Qvar 50 inhaler (Teva UK Ltd)
2,amp,3175611000001106,0302000C0BJABBF,Qvar 100 inhaler (Teva UK Ltd)
3,amp,3177411000001102,0302000C0BJACBG,Qvar 50 Autohaler (Teva UK Ltd)
4,amp,3177711000001108,0302000C0BJADBH,Qvar 100 Autohaler (Teva UK Ltd)
5,amp,8159511000001107,0302000C0BJAEBG,Qvar 50micrograms/dose Easi-Breathe inhaler (Teva UK Ltd)
6,amp,8159711000001102,0302000C0BJAFBH,Qvar 100micrograms/dose Easi-Breathe inhaler (Teva UK Ltd)
7,amp,9525111000001105,0302000C0BNAABU,Easyhaler Beclometasone 200micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)
8,amp,10617711000001103,0302000C0BPAABE,Clenil Modulite 50micrograms/dose inhaler (Chiesi Ltd)
9,amp,10618211000001109,0302000C0BPABBF,Clenil Modulite 100micrograms/dose inhaler (Chiesi Ltd)
10,amp,10619311000001107,0302000C0BPACBV,Clenil Modulite 200micrograms/dose inhaler (Chiesi Ltd)


## Combination Inhaled Corticosteroids

In [15]:
#Similar to above, to get combination products, we filter where there is more than one chemical substance in the inhaler
combi_ics = all_ics.loc[all_ics['chem_count'] >1]
combi_ics = combi_ics.drop(columns=["chem_count","strength"])
combi_ics

,type,id,bnf_code,nm
0,amp,34681611000001100,0301011ABBBAAA0,Trimbow 87micrograms/dose / 5micrograms/dose / 9micrograms/dose inhaler (Chiesi Ltd)
12,amp,12906411000001100,0302000C0BQAABX,Fostair 100micrograms/dose / 6micrograms/dose inhaler (Chiesi Ltd)
13,amp,31063411000001101,0302000C0BQABBZ,Fostair 200micrograms/dose / 6micrograms/dose inhaler (Chiesi Ltd)
14,amp,26112111000001106,0302000C0BRAABY,Fostair NEXThaler 100micrograms/dose / 6micrograms/dose dry powder inhaler (Chiesi Ltd)
15,amp,31063111000001106,0302000C0BRABCA,Fostair NEXThaler 200micrograms/dose / 6micrograms/dose dry powder inhaler (Chiesi Ltd)
25,amp,3294211000001101,0302000K0BDAAAL,Symbicort 100/6 Turbohaler (AstraZeneca UK Ltd)
26,amp,3294611000001104,0302000K0BDABAM,Symbicort 200/6 Turbohaler (AstraZeneca UK Ltd)
27,amp,4373811000001100,0302000K0BDACAU,Symbicort 400/12 Turbohaler (AstraZeneca UK Ltd)
28,amp,32926011000001100,0302000K0BDADBB,Symbicort 200micrograms/dose / 6micrograms/dose pressurised inhaler (AstraZeneca UK Ltd)
34,amp,25254111000001105,0302000K0BHAAAM,DuoResp Spiromax 160micrograms/dose / 4.5micrograms/dose dry powder inhaler (Teva UK Ltd)


## High Dose Corticosteroids

### Single agent High Dose Corticosteroids

In [16]:
#to get high dose single products, we have to filter where there is only one chemical substance in the inhaler AND where strength is "high"
single_high_ics = all_ics.loc[((all_ics['chem_count'] == 1) & (all_ics['strength'] == "high"))]
single_high_ics = single_high_ics.drop(columns=["chem_count","strength"])
single_high_ics

,type,id,bnf_code,nm
11,amp,10619611000001102,0302000C0BPADBW,Clenil Modulite 250micrograms/dose inhaler (Chiesi Ltd)
21,amp,36603811000001107,0302000C0BTADBW,Soprobec 250micrograms/dose inhaler (Glenmark Pharmaceuticals Europe Ltd)
23,amp,3228711000001106,0302000K0BBAIAH,Pulmicort 400 Turbohaler (AstraZeneca UK Ltd)
33,amp,10074411000001100,0302000K0BGACAH,Easyhaler Budesonide 400micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)
42,amp,3185211000001103,0302000N0BBAUAU,Flixotide 500micrograms/dose Accuhaler (GlaxoSmithKline UK Ltd)
44,amp,2831211000001109,0302000N0BBBABC,Flixotide 250micrograms/dose Evohaler (GlaxoSmithKline UK Ltd)
75,amp,9004211000001109,0302000U0BBABAB,Alvesco 160 inhaler (AstraZeneca UK Ltd)
93,amp,3179811000001108,0302000C0AAACAC,Beclometasone 250micrograms/dose inhaler (A A H Pharmaceuticals Ltd)
94,amp,3180011000001102,0302000C0AAACAC,Beclometasone 250micrograms/dose inhaler (Mylan)
95,amp,3180911000001103,0302000C0AAACAC,Beclometasone 250micrograms/dose inhaler (Alliance Healthcare (Distribution) Ltd)


### Combination High Dose Corticosteroids

In [17]:
#to get high dose single products, we have to filter where there is more than one chemical substance in the inhaler AND where strength is "high"
combi_high_ics = all_ics.loc[((all_ics['chem_count'] > 1) & (all_ics['strength'] == "high"))]
combi_high_ics = combi_high_ics.drop(columns=["chem_count","strength"])

combi_high_ics

,type,id,bnf_code,nm
13,amp,31063411000001101,0302000C0BQABBZ,Fostair 200micrograms/dose / 6micrograms/dose inhaler (Chiesi Ltd)
15,amp,31063111000001106,0302000C0BRABCA,Fostair NEXThaler 200micrograms/dose / 6micrograms/dose dry powder inhaler (Chiesi Ltd)
27,amp,4373811000001100,0302000K0BDACAU,Symbicort 400/12 Turbohaler (AstraZeneca UK Ltd)
35,amp,25254711000001106,0302000K0BHABAU,DuoResp Spiromax 320micrograms/dose / 9micrograms/dose dry powder inhaler (Teva UK Ltd)
36,amp,34812111000001106,0302000K0BIAAAU,Fobumix Easyhaler 320micrograms/dose / 9micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)
48,amp,3188311000001102,0302000N0BCACAZ,Seretide 500 Accuhaler (GlaxoSmithKline UK Ltd)
51,amp,539811000001106,0302000N0BCAFBG,Seretide 250 Evohaler (GlaxoSmithKline UK Ltd)
53,amp,21019711000001107,0302000N0BDABBK,Flutiform 250micrograms/dose / 10micrograms/dose inhaler (Napp Pharmaceuticals Ltd)
58,amp,29782511000001103,0302000N0BFABBG,Sirdupla 25micrograms/dose / 250micrograms/dose inhaler (Mylan)
59,amp,30950311000001106,0302000N0BGAAAZ,AirFluSal Forspiro 50micrograms/dose / 500micrograms/dose dry powder inhaler (Sandoz Ltd)


## Low/Medium Dose Inhaled Corticosteroids

### Combination Low/Medium Dose Corticosteroids

In [18]:
#to get low and medium dose combination products, we have to filter where there is more than one chemical substance in the inhaler AND where strength is "med_low"
combi_med_low_ics = all_ics.loc[((all_ics['chem_count'] >1) & (all_ics['strength'] == "med_low"))]
combi_med_low_ics = combi_med_low_ics.drop(columns=["chem_count","strength"])
combi_med_low_ics

,type,id,bnf_code,nm
0,amp,34681611000001100,0301011ABBBAAA0,Trimbow 87micrograms/dose / 5micrograms/dose / 9micrograms/dose inhaler (Chiesi Ltd)
12,amp,12906411000001100,0302000C0BQAABX,Fostair 100micrograms/dose / 6micrograms/dose inhaler (Chiesi Ltd)
14,amp,26112111000001106,0302000C0BRAABY,Fostair NEXThaler 100micrograms/dose / 6micrograms/dose dry powder inhaler (Chiesi Ltd)
25,amp,3294211000001101,0302000K0BDAAAL,Symbicort 100/6 Turbohaler (AstraZeneca UK Ltd)
26,amp,3294611000001104,0302000K0BDABAM,Symbicort 200/6 Turbohaler (AstraZeneca UK Ltd)
28,amp,32926011000001100,0302000K0BDADBB,Symbicort 200micrograms/dose / 6micrograms/dose pressurised inhaler (AstraZeneca UK Ltd)
34,amp,25254111000001105,0302000K0BHAAAM,DuoResp Spiromax 160micrograms/dose / 4.5micrograms/dose dry powder inhaler (Teva UK Ltd)
37,amp,34950311000001108,0302000K0BIABAM,Fobumix Easyhaler 160micrograms/dose / 4.5micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)
38,amp,34950611000001103,0302000K0BIADAL,Fobumix Easyhaler 80micrograms/dose / 4.5micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)
46,amp,3186911000001100,0302000N0BCAAAX,Seretide 100 Accuhaler (GlaxoSmithKline UK Ltd)


### Single agent Low/Medium Dose Corticosteroids

In [19]:
#to get low and medium dose single products, we have to filter where there is only chemical substance in the inhaler AND where strength is "med_low"
single_med_low_ics = all_ics.loc[((all_ics['chem_count'] == 1) & (all_ics['strength'] == "med_low"))]
single_med_low_ics = single_med_low_ics.drop(columns=["chem_count","strength"])
single_med_low_ics

,type,id,bnf_code,nm
1,amp,3176811000001102,0302000C0BJAABE,Qvar 50 inhaler (Teva UK Ltd)
2,amp,3175611000001106,0302000C0BJABBF,Qvar 100 inhaler (Teva UK Ltd)
3,amp,3177411000001102,0302000C0BJACBG,Qvar 50 Autohaler (Teva UK Ltd)
4,amp,3177711000001108,0302000C0BJADBH,Qvar 100 Autohaler (Teva UK Ltd)
5,amp,8159511000001107,0302000C0BJAEBG,Qvar 50micrograms/dose Easi-Breathe inhaler (Teva UK Ltd)
6,amp,8159711000001102,0302000C0BJAFBH,Qvar 100micrograms/dose Easi-Breathe inhaler (Teva UK Ltd)
7,amp,9525111000001105,0302000C0BNAABU,Easyhaler Beclometasone 200micrograms/dose dry powder inhaler (Orion Pharma (UK) Ltd)
8,amp,10617711000001103,0302000C0BPAABE,Clenil Modulite 50micrograms/dose inhaler (Chiesi Ltd)
9,amp,10618211000001109,0302000C0BPABBF,Clenil Modulite 100micrograms/dose inhaler (Chiesi Ltd)
10,amp,10619311000001107,0302000C0BPACBV,Clenil Modulite 200micrograms/dose inhaler (Chiesi Ltd)


### Create codelists

In [21]:
#Here we create all codelists as csvs
single_ics.to_csv(os.path.join('..','data','new_single_ics_codelist.csv'))
combi_ics.to_csv(os.path.join('..','data','new_combi_ics_codelist.csv'))
combi_high_ics.to_csv(os.path.join('..','data','new_combi_high_dose_ics_codelist.csv'))
single_high_ics.to_csv(os.path.join('..','data','new_single_high_dose_ics_codelist.csv'))
combi_med_low_ics.to_csv(os.path.join('..','data','new_combi_med_low_ics_codelist.csv'))
single_med_low_ics.to_csv(os.path.join('..','data','new_single_med_low_dose_ics_codelist.csv'))